In [11]:
import pandas as pd
import numpy as np
from osgeo import gdal, osr, ogr
import os
import sys
import math
from matplotlib import pyplot as plt

In [12]:
SPATIAL_IDX = 5
TEMPORAL_IDX = 9

In [13]:
'''
Open input file <filename> and return numpy array
'''

def getModisData(filename):
    # This allows gdal to throw python exceptions
    gdal.UseExceptions()
    try:
        modisRaster = gdal.Open(filename)
    except e:
        print("Unable to open tif file")
        print(e)
        print(modisRaster.GetMetadata())
    modisData = modisRaster.ReadAsArray()
    # set NaN values to 0
    modisData = np.where((modisData < 1) | (modisData > 366), 0, modisData)
    modisData = np.floor(modisData)
    return modisData,modisRaster
    

In [14]:
'''
Set the number of spatial window based on row/col and SPATIAL_IDX
'''
def setSpatialWindow(row,col,arrayDims):
    top = max(0,row-SPATIAL_IDX)
    bottom = min(arrayDims[0],row+SPATIAL_IDX+1)
    left = max(0,col-SPATIAL_IDX)
    right = min(arrayDims[1],col+SPATIAL_IDX+1)
    return top,bottom,left,right
    

In [15]:
'''
Process the input array determining fire events based on SPATIAL_IDX and TEMPORAL_IDX
'''
def getEventData(inputArray):
    nextId = 1
    eventArray = np.zeros(inputArray.shape) #initialize event array to 0s
    #Traverse pixels in the MODIS Data Array to look for neighboring event pixels that belong to the event
    for row in range(inputArray.shape[0]):
        for col in range(inputArray.shape[1]):
            if inputArray[row,col] : # burn detected
                #print("{},{} burn detected".format(row,col))
                burnDay = inputArray[row,col]
                #print("Next ID={}, burnDay={}:".format(nextId,burnDay))
                # set spatial input and temporal bounds and create spatial windows
                top,bottom,left,right = setSpatialWindow(row,col,inputArray.shape)
                inputWindow = inputArray[top:bottom,left:right]
                temporalMask = np.logical_and(inputWindow>0,abs(inputWindow-burnDay)<=TEMPORAL_IDX)
                eventWindow = eventArray[top:bottom,left:right]
                
                currId = eventArray[row,col] # Event ID for current pixel
                overlapMask = np.logical_and(np.logical_and(eventWindow>0,temporalMask),eventWindow!=currId)
                if overlapMask.any():
                    #true overlap
                    overlapId = np.amin(eventWindow[overlapMask],axis=None)
                    eventWindow[temporalMask] = overlapId
                    if currId != 0:
                        #print("Event merge: setting {} to {}".format(currId,overlapId))
                        eventArray[eventArray==currId] = overlapId
                elif currId == 0: # no overlap and not set so get next ID
                    eventWindow[temporalMask] = nextId
                    nextId += 1
                else: 
                    eventWindow[temporalMask] = currId # propagate current id
    print(eventArray)
    return eventArray

In [16]:
'''
Save the event data to GTiff file
'''
def saveEventData(eventData,input_raster,outfile):
    rows = input_raster.RasterYSize
    cols = input_raster.RasterXSize
    output_raster = gdal.GetDriverByName('GTiff').Create(outfile, cols, rows, 1 ,gdal.GDT_Float32)
    output_raster.SetGeoTransform(input_raster.GetGeoTransform())
    output_raster.SetProjection(input_raster.GetProjection())
    output_raster.GetRasterBand(1).WriteArray(eventData) 

In [17]:
input_path = '../data/MCD64A1/C6/yearly_composites/'
out_path = '../data/MCD64A1/C6/yearly_events/'
vars = ['raw', 'filled', 'cleaned']

for i in range(2001,2016) :
    for j in vars :
        if not os.path.exists(out_path + 'USA_burnevents_' + str(i) + '_' + j + '.tif'):
            modisData,modisRaster = getModisData(input_path + 'USA_BurnDate_' + str(i) + '_' + j + '.tif')
            eventData = getEventData(modisData)
            saveEventData(eventData,modisRaster, out_path + 'USA_burnevents_' + str(i) + '_' + j + '.tif')

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

In [26]:
eventData.dtype

dtype('float64')

Just a quick check to make sure the saved data matches the event data. You can delete.

In [58]:
savedRaster = gdal.Open('../out/waldo_raw_out.tif')
savedData = savedRaster.ReadAsArray()
print(np.count_nonzero(eventData))
print(np.count_nonzero(savedData))
print(np.count_nonzero(eventData-savedData))

335
335
0


In [59]:
savedData.max()

32.0

In [30]:
!gdalinfo -stats ../out/WUS_BurnDate_2014_out.tif

Driver: GTiff/GeoTIFF
Files: ../out/WUS_BurnDate_2014_out.tif
Size is 3720, 4320
Coordinate System is:
PROJCS["unnamed",
    GEOGCS["unnamed ellipse",
        DATUM["unknown",
            SPHEROID["unnamed",6370997,0]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433]],
    PROJECTION["Lambert_Azimuthal_Equal_Area"],
    PARAMETER["latitude_of_center",45],
    PARAMETER["longitude_of_center",-100],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]]]
Origin = (-2032117.735286616021767,681214.871921156998724)
Pixel Size = (500.000000000000000,-500.000000000000000)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (-2032117.735,  681214.872) (127d57' 2.67"W, 47d52'11.96"N)
Lower Left  (-2032117.735,-1478785.128) (121d 2'16.90"W, 29d25' 6.64"N)
Upper Right ( -172117.735,  681214.872) (102d27'43.99"W, 51d 6'18.20"N)
Lower Right ( -172117.73